# Emergency
Calculate what your emergency fund size should be along with other useful
  things related to it.

In [ ]:
# | default_exp emergency


### Set your `YNAB_TOKEN` environment variable
We'll need a token to access our YNAB account.  To get this you'll need to go
into your YNAB account and get one.  Then set it as an environment variable.

In [4]:
import os

token = os.getenv("YNAB_TOKEN")

### Pull down some transactions

In [ ]:
# | export
from budgeting_tools.budget_utils import (
    get_transactions,
    get_avg_monthly_spend,
    get_cash_balance,
    get_budgeted_balance,
)


In [7]:
trans_df = get_transactions(token, 3)

In [8]:
trans_df.columns


Index(['id', 'date', 'amount', 'memo', 'cleared', 'approved', 'flag_color',
       'account_id', 'account_name', 'payee_id', 'payee_name', 'category_id',
       'category_name', 'transfer_account_id', 'transfer_transaction_id',
       'matched_transaction_id', 'import_id', 'import_payee_name',
       'import_payee_name_original', 'debt_transaction_type', 'deleted',
       'subtransactions', 'month', 'transaction_id'],
      dtype='object')

In [14]:
essential_categories = [
    "🎉Fun Money",
    "🍎Groceries",
    "🏋️Health",
    "🏦Insurance",
    "☎️Phone",
    "🚘Transportation",
    "⚡️Utilities",
    "🏡Mortgage",
]
monthly_spend = get_avg_monthly_spend(trans_df, essential_categories)


### Get emergency fund size
To get our emergency fund size, we'll just take our average monthly spend on
essential categories and multiply it by the number of months we want to have.

In [32]:
emergency_fund_months = 6
emergency_fund_size = monthly_spend * emergency_fund_months
print(f"Our emergency fund should be ${emergency_fund_size*-1:,.2f}")

Our emergency fund should be $26,295.24


### Make sure we have enough in our emergency fund.  Not too much, not too little.

In [31]:
cash = get_cash_balance(token)
budgeted = get_budgeted_balance(token)
# print cash and budgeted and add commas to the numbers
print(f"Our total cash balance is ${cash:,.2f}")
print(f"Our total budgeted balance is ${budgeted:,.2f}")

Our total cash balance is $155,571.50
Our total budgeted balance is $16,700.95


Lets make see how much more we need to save to get to our emergency fund size,
or how much more we have than we need and should probably invest.

In [36]:
unbudgeted_cash = cash - budgeted
non_emergency_cash = unbudgeted_cash - (emergency_fund_size * -1)

print(f"Unbudgeted cash is ${unbudgeted_cash:,.2f}")
if non_emergency_cash >= 0:
    print(
        f"Fully funded emergency fund.  Move ${non_emergency_cash:,.2f} to a high yield savings account or investment account."
    )
else:
    print(f"Save ${non_emergency_cash:,.2f} more to fully fund an emergency fund")

Unbudgeted cash is $138,870.55
Fully funded emergency fund.  Move $112,575.31 to a high yield savings account or investment account.


In [38]:
# | export
def get_emergency_fund_status(
    token, essential_categories, n_months=6, emergency_fund_months=6
):
    """Function to get the status of the emergency fund.  Returns the amount
    needed to fully fund the emergency fund or the amount of non-emergency cash
    if the emergency fund is fully funded"""
    trans_df = get_transactions(token, n_months)
    monthly_spend = get_avg_monthly_spend(trans_df, essential_categories)
    cash = get_cash_balance(token)
    budgeted = get_budgeted_balance(token)
    emergency_fund_size = monthly_spend * emergency_fund_months
    unbudgeted_cash = cash - budgeted
    non_emergency_cash = unbudgeted_cash - (emergency_fund_size * -1)
    return non_emergency_cash

In [39]:
get_emergency_fund_status(token, essential_categories, 6, 6)


111786.6